In [1]:
import torch
import numpy as np
import sympy as sp
import os, sys
import odeformer
import requests
from IPython.display import display

In [2]:
model_path = "model.pt" 
try:
    if not os.path.isfile(model_path): 
        url = "https://dl.fbaipublicfiles.com/odeformer/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model = torch.load(model_path)
        model = model.cuda()
    print(model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)    

cuda:0
Model successfully loaded!


In [3]:
est = odeformer.model.SymbolicTransformerRegressor(
                        model=model,
                        max_input_points=200,
                        n_trees_to_refine=100,
                        rescale=True
                        )

In [4]:
##Example of data

x = np.random.randn(100, 2)
y = np.cos(2*np.pi*x[:,0])+x[:,1]**2


In [5]:
est.fit(x,y)
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
model_str = est.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    model_str = model_str.replace(op,replace_op)
display(sp.parse_expr(model_str))

/home/dascoli/miniconda3/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


0.89887978783496991*(x_1 - 0.010476798276570112)**2 - 0.9760000000000001*sin(5.591313448981332*x_0 + 4.3736305073537418) + 0.05578000000000001

In [12]:
from odeformer.envs.generators import RandomFunctions

generator = RandomFunctions()